In [8]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(1)
torch.cuda.manual_seed_all(1)
#====================================================================
# calling data
trans = transforms.Compose([transforms.Resize((100,100)),
                           transforms.ToTensor(),
                           transforms.Normalize((-1,-1,-1),(2,2,2))])
trainset = torchvision.datasets.ImageFolder(root="/home/gus/catvsdog",
                                           transform = trans)
train_loader = torch.utils.data.DataLoader(trainset,
                                          batch_size = 100,
                                          shuffle=True)
#====================================================================
# declaring hyperparameters
lr = 0.01
#====================================================================
# constructiong CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        # input shape = (?, 100, 100, 3)
        # Conv -> (?, 100, 100, 16)
        # Pool -> (?, 50, 50, 16)
        self.layer = nn.Sequential(
            torch.nn.Conv2d(3,16,3,stride=1,padding=1),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2),
        # input shape = (?, 50, 50, 16)
        # Conv -> (?, 50, 50, 32)
        # Pool -> (?, 25, 25, 32)
            torch.nn.Conv2d(16,32,3,stride=1,padding=1),
            torch.nn.BatchNorm2d(32),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.fc = torch.nn.Linear(25*25*32,2,bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
    
    def forward(self,x):
        out = self.layer(x)
        out = out.view(out.size(0),-1)
        out = self.fc(out)
        return out
#====================================================================
model=CNN().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.RMSprop(model.parameters(),lr = lr)

total_batch = len(train_loader)

for epoch in range(10):
    avg_cost = 0
    for X,Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost/total_batch
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 32.5344925
[Epoch:    2] cost = 2.60450292
[Epoch:    3] cost = 1.23256409
[Epoch:    4] cost = 0.692862689
[Epoch:    5] cost = 0.635398626
[Epoch:    6] cost = 0.602166295
[Epoch:    7] cost = 0.512836754
[Epoch:    8] cost = 0.631494999
[Epoch:    9] cost = 0.463401973
[Epoch:   10] cost = 0.436750054


In [10]:
# calling data
trans = transforms.Compose([transforms.Resize((100,100)),
                           transforms.ToTensor(),
                           transforms.Normalize((-1,-1,-1),(2,2,2))])
testset = torchvision.datasets.ImageFolder(root="/home/gus/cattest",
                                           transform = trans)
test_loader = torch.utils.data.DataLoader(testset,
                                          batch_size = 100,
                                          shuffle=True)
with torch.no_grad():
    for X,Y in test_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        prediction = model(X)
        correct_prediction = torch.argmax(prediction,1) == Y
        accuracy = correct_prediction.float().mean()
        print('Accuracy:',accuracy.item())


Accuracy: 0.6299999952316284
Accuracy: 0.6899999976158142
Accuracy: 0.7599999904632568
Accuracy: 0.6899999976158142
Accuracy: 0.699999988079071
Accuracy: 0.6899999976158142
Accuracy: 0.7400000095367432
Accuracy: 0.7199999690055847
Accuracy: 0.6299999952316284
Accuracy: 0.7299999594688416
Accuracy: 0.7799999713897705
Accuracy: 0.7099999785423279
Accuracy: 0.7199999690055847
Accuracy: 0.6899999976158142
Accuracy: 0.7099999785423279
Accuracy: 0.6299999952316284
Accuracy: 0.6299999952316284
Accuracy: 0.6200000047683716
Accuracy: 0.6200000047683716
Accuracy: 0.6699999570846558
